In [5]:
# Import required Libraries
import subprocess
import sys
import os
from pathlib import Path

def install_module(module_name):
    try:
        __import__(module_name)
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

# List of modules to check and install if necessary
modules = ['h5py', 'numpy', 'requests', 'geopandas', 'matplotlib', 'pandas', 'pyproj', 'shapely', 'xarray', 'rasterio', 'tqdm']
for module in modules:
    install_module(module)

# Import the rest of the required libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
import logging
from pathlib import Path
import rasterio
from rasterio.plot import show


In [ ]:
# Flexible Import for RAS Commander
import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import (init_ras_project, HdfBase, HdfUtils, HdfFluvialPluvial, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, HdfResultsXsec, HdfPipe, HdfPump, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras)
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger
except ImportError:
    # If the import fails, add the parent directory to the Python path
    print("Using Local Dev Copy")
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import (init_ras_project, HdfBase, HdfUtils, HdfFluvialPluvial, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, HdfResultsXsec, HdfPipe, HdfPump, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras)
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger

print("ras_commander imported successfully")

# Example of a Terrains section of a RASMapper project file
  
  
  
  <Terrains Checked="True" Expanded="True">
    <Layer Name="Terrain50" Type="TerrainLayer" Checked="True" Filename=".\Terrain\Terrain50.hdf">
      <Symbology>
        <SurfaceFill Colors="-10039894,-256,-16744448,-23296,-7667712,-5952982,-8355712,-1286" Values="465.59375,795.351225457753,1044.36749240828,1242.53287192629,1435.70761903517,1620.52302566217,1835.85718004039,2542.21875" Stretched="True" AlphaTag="255" UseDatasetMinMax="False" RegenerateForScreen="False" />
      </Symbology>
      <ResampleMethod>near</ResampleMethod>
      <Surface On="True" />
    </Layer>
  </Terrains>

In [7]:
terrain_hdf_path = r"C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\Terrain\Terrain50.hdf"

In [ ]:
HdfBase.get_dataset_info(terrain_hdf_path, "/")

In [ ]:
import h5py
import os
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show

# Enable GDAL exceptions to avoid FutureWarning
gdal.UseExceptions()

def get_tiff_filenames_from_hdf(hdf_file):
    """
    Extracts TIFF filenames from the specified HDF5 file.

    Args:
        hdf_file (str): Path to the HDF5 file.

    Returns:
        list of str: List of extracted TIFF filenames.
    """
    tiff_files = []
    with h5py.File(hdf_file, 'r') as f:
        terrain_group = f.get('/Terrain')
        if terrain_group:
            for item in terrain_group:
                item_path = f'/Terrain/{item}'
                group = f.get(item_path)
                if group and 'File' in group.attrs:
                    tiff_filename = group.attrs['File'].decode('utf-8')
                    tiff_files.append(tiff_filename)
    return tiff_files

def get_tiff_properties(tiff_file):
    """
    Retrieves properties of the specified TIFF file.

    Args:
        tiff_file (str): Path to the TIFF file.

    Returns:
        dict: Properties of the TIFF file including resolution and elevation range.
    """
    ds = gdal.Open(tiff_file)
    if ds is None:
        print(f"Could not open {tiff_file}")
        return None
    
    x_size, y_size = ds.RasterXSize, ds.RasterYSize
    geotransform = ds.GetGeoTransform()
    
    # Calculate extents
    x_min = geotransform[0]
    x_max = x_min + geotransform[1] * x_size
    y_max = geotransform[3] 
    y_min = y_max + geotransform[5] * y_size
    
    # Get elevation range
    band = ds.GetRasterBand(1)
    nodata = band.GetNoDataValue()
    data = band.ReadAsArray()
    data = data[data != nodata]  # Remove nodata values
    elev_min = np.min(data) if len(data) > 0 else None
    elev_max = np.max(data) if len(data) > 0 else None
    elev_range = elev_max - elev_min if (elev_min is not None and elev_max is not None) else 0
    
    extent_width = abs(x_max - x_min)
    extent_height = abs(y_max - y_min)
    
    # Calculate resolution (area per pixel)
    pixel_width, pixel_height = abs(geotransform[1]), abs(geotransform[5])
    resolution = pixel_width * pixel_height
    
    projection = ds.GetProjection()
    ds = None
    
    return {
        'filename': tiff_file,
        'x_size': x_size,
        'y_size': y_size,
        'pixel_width': pixel_width,
        'pixel_height': pixel_height,
        'resolution': resolution,
        'x_min': x_min,
        'x_max': x_max,
        'y_min': y_min,
        'y_max': y_max,
        'extent_width': extent_width,
        'extent_height': extent_height,
        'elev_min': elev_min,
        'elev_max': elev_max,
        'elev_range': elev_range,
        'projection': projection
    }

def get_priority_tiff(tiff_files):
    """
    Identifies the TIFF with largest resolution (lowest point density).

    Args:
        tiff_files (list of str): List of TIFF filenames.

    Returns:
        tuple: (priority TIFF properties, list of all TIFF properties)
    """
    tiff_properties_list = [get_tiff_properties(tiff_file) for tiff_file in tiff_files]
    tiff_properties_list = [props for props in tiff_properties_list if props]
    # Sort by resolution (largest resolution first)
    tiff_properties_list.sort(key=lambda x: x['resolution'], reverse=True)
    return tiff_properties_list[0], tiff_properties_list

def process_tiffs(priority_tiff_props, tiff_properties_list):
    """
    Processes TIFF files by starting with priority TIFF and updating its values
    where other TIFFs have lower elevations within their valid masks.

    Args:
        priority_tiff_props (dict): Properties of the priority TIFF.
        tiff_properties_list (list of dict): Properties of all TIFFs.
    """
    # Open priority TIFF and get its data
    priority_ds = gdal.Open(priority_tiff_props['filename'])
    priority_array = priority_ds.GetRasterBand(1).ReadAsArray().astype(np.float32)
    priority_nodata = priority_ds.GetRasterBand(1).GetNoDataValue()
    priority_mask = priority_array != priority_nodata
    
    # Get geotransform and projection from priority TIFF
    priority_geotransform = priority_ds.GetGeoTransform()
    priority_projection = priority_ds.GetProjection()
    x_size, y_size = priority_ds.RasterXSize, priority_ds.RasterYSize
    
    # Initialize output array with priority TIFF data
    output_array = priority_array.copy()
    output_mask = priority_mask.copy()
    
    # Process each additional TIFF
    for tiff_props in tiff_properties_list[1:]:
        print(f"Processing {tiff_props['filename']}")
        
        # Open current TIFF
        ds = gdal.Open(tiff_props['filename'])
        if ds is None:
            continue
            
        # Create memory dataset for resampling
        mem_driver = gdal.GetDriverByName('MEM')
        mem_ds = mem_driver.Create('', x_size, y_size, 1, gdal.GDT_Float32)
        mem_ds.SetGeoTransform(priority_geotransform)
        mem_ds.SetProjection(priority_projection)
        
        # Resample current TIFF to match priority TIFF
        gdal.Warp(mem_ds, ds, format='MEM',
                  resampleAlg='bilinear',
                  xRes=priority_geotransform[1],
                  yRes=-priority_geotransform[5],
                  outputBounds=(priority_geotransform[0],
                              priority_geotransform[3] + priority_geotransform[5] * y_size,
                              priority_geotransform[0] + priority_geotransform[1] * x_size,
                              priority_geotransform[3]))
        
        # Get resampled data and its mask
        current_array = mem_ds.GetRasterBand(1).ReadAsArray()
        current_nodata = mem_ds.GetRasterBand(1).GetNoDataValue()
        current_mask = current_array != current_nodata
        
        # Only update where:
        # 1. Current TIFF has valid data AND
        # 2. Priority TIFF has no data
        update_mask = np.logical_and(current_mask, ~priority_mask)
        
        # Update output array only in areas where priority TIFF has no data
        output_array[update_mask] = current_array[update_mask]
        output_mask[update_mask] = True
        
        ds = None
        mem_ds = None
    
    # Apply final nodata mask
    output_array[~output_mask] = priority_nodata
    
    # Save output
    output_file = 'merged_terrain.tif'
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_file, x_size, y_size, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(priority_geotransform)
    out_ds.SetProjection(priority_projection)
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(output_array)
    out_band.SetNoDataValue(priority_nodata)
    out_ds.FlushCache()
    out_ds = None
    
    print(f"Output saved to {output_file}")
    
    # Plot the output TIFF
    fig, ax = plt.subplots(figsize=(10, 10))
    with rasterio.open(output_file) as src:
        img = show(src, ax=ax, title='Merged Terrain')
        plt.colorbar(img.get_images()[0], ax=ax, label='Elevation')
    plt.show()

def main():
    hdf_file = 'C:\\GH\\ras-commander\\examples\\example_projects\\BaldEagleCrkMulti2D\\Terrain\\Terrain50.hdf'
    tiff_files = get_tiff_filenames_from_hdf(hdf_file)
    hdf_dir = os.path.dirname(hdf_file)
    tiff_files = [os.path.join(hdf_dir, tiff_file) for tiff_file in tiff_files]
    
    priority_tiff, tiff_properties_list = get_priority_tiff(tiff_files)
    print(f"Priority TIFF is {priority_tiff['filename']}")
    print(f"Resolution: {priority_tiff['resolution']:.2f} square units per pixel")
    print(f"Pixel dimensions: {priority_tiff['pixel_width']:.2f} x {priority_tiff['pixel_height']:.2f}")
    
    process_tiffs(priority_tiff, tiff_properties_list)

if __name__ == "__main__":
    main()


In [ ]:
def visualize_tiffs(hdf_file):
    """
    Creates plots of original TIFFs without the final merged TIFF.
    """
    import matplotlib.pyplot as plt
    import rasterio
    from rasterio.plot import show
    
    # Get TIFF files
    tiff_files = get_tiff_filenames_from_hdf(hdf_file)
    hdf_dir = os.path.dirname(hdf_file)
    tiff_files = [os.path.join(hdf_dir, tiff_file) for tiff_file in tiff_files]
    
    # Get properties
    priority_tiff, tiff_properties_list = get_priority_tiff(tiff_files)
    
    # Calculate grid size
    n_plots = len(tiff_files)
    n_rows = (n_plots + 1) // 2
    n_cols = min(2, n_plots)
    
    # Create figure
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 7*n_rows))
    if n_rows == 1 and n_cols == 1:
        axes = np.array([axes])
    axes = axes.flatten()
    
    # Plot original TIFFs
    for idx, tiff_file in enumerate(tiff_files):
        try:
            with rasterio.open(tiff_file) as src:
                show(src, ax=axes[idx], title=f'Original TIFF: {os.path.basename(tiff_file)}')
                
                props = tiff_properties_list[idx]
                info_text = f"Size: {props['x_size']}x{props['y_size']}\n"
                info_text += f"Resolution: {abs(props['pixel_width']):.2f}x{abs(props['pixel_height']):.2f}"
                
                axes[idx].text(0.02, 0.98, info_text,
                             transform=axes[idx].transAxes,
                             bbox=dict(facecolor='white', alpha=0.8),
                             verticalalignment='top')
        except Exception as e:
            print(f"Error plotting {tiff_file}: {str(e)}")
    
    # Remove any extra empty subplots
    for idx in range(n_plots, len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    plt.show()
    
    # Print summary information
    print("\nTerrain Files Summary:")
    print("-" * 50)
    for idx, props in enumerate(tiff_properties_list):
        print(f"\nTIFF {idx+1}:")
        print(f"Filename: {os.path.basename(props['filename'])}")
        print(f"Dimensions: {props['x_size']} x {props['y_size']}")
        print(f"Resolution: {abs(props['pixel_width']):.2f} x {abs(props['pixel_height']):.2f}")

# Run the visualization
hdf_file = r"C:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\Terrain\Terrain50.hdf"
visualize_tiffs(hdf_file)

In [ ]:
# To resolve the ModuleNotFoundError for 'osgeo', we need to install the GDAL package.
# This can be done using conda. The following command should be run in the terminal or command prompt:

# Install GDAL using conda
#!conda install -c conda-forge gdal -y

# After installation, we can proceed with the import statement.
# Make sure to restart the kernel if necessary to recognize the newly installed package.


In [ ]:
# Download the BaldEagleCrkMulti2D project from HEC and Run Plan 06

# Define the path to the BaldEagleCrkMulti2D project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
bald_eagle_path = current_dir / "example_projects" / "BaldEagleCrkMulti2D"
import logging

# Check if BaldEagleCrkMulti2D.p06.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = bald_eagle_path / "BaldEagleDamBrk.p06.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the BaldEagleCrkMulti2D project
    ras_examples = RasExamples()
    ras_examples.extract_project(["BaldEagleCrkMulti2D"])

    # Initialize custom Ras object
    bald_eagle = RasPrj()

    # Initialize the RAS project using the custom ras object
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    logging.info(f"Bald Eagle project initialized with folder: {bald_eagle.project_folder}")
    
    logging.info(f"Bald Eagle object id: {id(bald_eagle)}")
    
    # Define the plan number to execute
    plan_number = "06"

    # Update run flags for the project
    RasPlan.update_run_flags(
        plan_number,
        geometry_preprocessor=True,
        unsteady_flow_simulation=True,
        run_sediment=False,
        post_processor=True,
        floodplain_mapping=False,
        ras_object=bald_eagle
    )

    # Execute Plan 06 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_bald_eagle = RasCmdr.compute_plan(plan_number, ras_object=bald_eagle)
    if success_bald_eagle:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("BaldEagleCrkMulti2D.p06.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the custom ras object
    bald_eagle = RasPrj()
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    plan_number = "06"

In [ ]:
# Load Plan and Geometry Dataframes and find Plan and Geometry HDF Paths

# Display plan_df for bald_eagle project
print("Plan DataFrame for bald_eagle project:")
display(bald_eagle.plan_df)

# Display geom_df for bald_eagle project
print("\nGeometry DataFrame for bald_eagle project:")
display(bald_eagle.geom_df)

# Get the plan HDF path
plan_number = "06"  # Assuming we're using plan 01 as in the previous code
plan_hdf_path = bald_eagle.plan_df.loc[bald_eagle.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]


In [ ]:
# Example: Get mesh maximum water surface elevation
max_ws_df = HdfResultsMesh.get_mesh_max_ws(plan_hdf_path, ras_object=bald_eagle)
print("\nMesh Maximum Water Surface Elevation:")
print(max_ws_df.attrs)
display(max_ws_df.head())